# Bangkit 2022 Capstone Project
This project aims to classify the sentiment of a text as either positive or negative. It involves transfer learning using IndoBERT. The data collection is done through a combination of semi-manual scraping, automated scraping, and open data from the internet.

In [48]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertModel

### Training data preprocessing:

In [49]:
df_gmaps = pd.read_csv("./data/google_maps.csv")
df_tokped = pd.read_csv("./data/dataset_review_tokped_labelled.csv")

df_tokped.head()

,Review,Rating,Sentiment
0,enak kuacinya,5,positive
1,pengiriman cepat packing bagus sesuai pesanan ...,5,positive
2,pengemasan luar biasa baik untuk rasa menurut ...,4,negative
3,terimakasih min,5,neutral
4,udah order untuk kesekian kali jos,5,neutral


In [50]:
df_tokped.drop("Rating", axis="columns", inplace=True)
df_tokped.head()

,Review,Sentiment
0,enak kuacinya,positive
1,pengiriman cepat packing bagus sesuai pesanan ...,positive
2,pengemasan luar biasa baik untuk rasa menurut ...,negative
3,terimakasih min,neutral
4,udah order untuk kesekian kali jos,neutral


In [51]:
df_tokped.columns = ["text", "label"]
df_tokped.head()

,text,label
0,enak kuacinya,positive
1,pengiriman cepat packing bagus sesuai pesanan ...,positive
2,pengemasan luar biasa baik untuk rasa menurut ...,negative
3,terimakasih min,neutral
4,udah order untuk kesekian kali jos,neutral


In [52]:
# change "positive" or "neutral" to 1, change "negative" to 0
df_tokped["label"] = df_tokped["label"].map(lambda row: 0 if row == "negative" else 1)
df_tokped.head()

,text,label
0,enak kuacinya,1
1,pengiriman cepat packing bagus sesuai pesanan ...,1
2,pengemasan luar biasa baik untuk rasa menurut ...,0
3,terimakasih min,1
4,udah order untuk kesekian kali jos,1


In [53]:
df_tokped["label"].value_counts()

label
1    3488
0     572
Name: count, dtype: int64

In [54]:
df = pd.concat([df_gmaps, df_tokped], ignore_index=True)

df.head()

,text,label
0,Tempat yang enak untuk hang out bersama teman ...,1
1,Tempatnya nyaman krn smoking areanya benar2 te...,1
2,Tempat ternyaman dan deket banget sama kantor....,1
3,"Tempatnya luas bgtt, nyaman kalo buat nugas ku...",1
4,Tempatnya cukup luas. Bisa blocking. Instagram...,1


### Train test split

In [55]:
# shuffle training data
df = df.sample(frac=1, ignore_index=True)

df.head()

,text,label
0,gak pernah mengecewakan,1
1,uenak tenan,1
2,tiap bulan beli produk ini selalu konsisten ku...,1
3,kemasan bagus rapi n aman jadi langganan terussss,1
4,cepet banget nyampenya dari bdg ke bekasi rasa...,1


In [56]:
# train-valid-test split 70-20-10
train_size = int(len(df) * 0.7)
valid_size = int(len(df) * 0.2)

df_train = df[:train_size]
df_valid = df[train_size:train_size + valid_size]
df_test = df[train_size + valid_size:]

print(len(df_train))
print(len(df_valid))
print(len(df_test))

2982
852
426


In [57]:
x_train = df_train["text"].values
y_train = df_train["label"].values

x_valid = df_valid["text"].values
y_valid = df_valid["label"].values

x_test = df_test["text"].values
y_test = df_test["label"].values

### Modelling

In [58]:
# download the IndoBERT pre-trained model
model_name='cahya/bert-base-indonesian-522M'
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)
bert_model.trainable = False

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [59]:
# tokenize the training data using bert tokenizer
x_train_tokenized = bert_tokenizer(x_train.tolist(), truncation=True, max_length=100, padding=True, return_tensors="tf")
x_valid_tokenized = bert_tokenizer(x_valid.tolist(), truncation=True, max_length=100, padding=True, return_tensors="tf")
x_test_tokenized = bert_tokenizer(x_test.tolist(), truncation=True, max_length=100, padding=True, return_tensors="tf")

In [60]:
# define the model layers
input_layer = tf.keras.Input(shape=(x_train_tokenized.input_ids.shape[1],), dtype=tf.int32, name="input_ids")
bert_output = bert_model(input_layer)
last_hidden_states = bert_output.last_hidden_state
hidden_layer = tf.keras.layers.Dense(128, activation='relu')(last_hidden_states[:, 0, :])
classification_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer)

model = tf.keras.models.Model(inputs=input_layer, outputs=classification_layer)

In [61]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [64]:
model.fit(x_train_tokenized.input_ids, y_train, epochs=10, batch_size=100, validation_data=(x_valid_tokenized.input_ids, y_valid))

Epoch 1/10
30/30 [==============================] - 402s 13s/step - loss: 0.4057 - accuracy: 0.8414 - val_loss: 0.3562 - val_accuracy: 0.8592
Epoch 2/10
30/30 [==============================] - 380s 13s/step - loss: 0.3947 - accuracy: 0.8427 - val_loss: 0.3499 - val_accuracy: 0.8627
Epoch 3/10
30/30 [==============================] - 400s 13s/step - loss: 0.3838 - accuracy: 0.8474 - val_loss: 0.3729 - val_accuracy: 0.8615
Epoch 4/10
30/30 [==============================] - 434s 15s/step - loss: 0.3686 - accuracy: 0.8491 - val_loss: 0.3296 - val_accuracy: 0.8638
Epoch 5/10
30/30 [==============================] - 428s 14s/step - loss: 0.3513 - accuracy: 0.8555 - val_loss: 0.3236 - val_accuracy: 0.8685
Epoch 6/10
30/30 [==============================] - 459s 15s/step - loss: 0.3480 - accuracy: 0.8595 - val_loss: 0.3606 - val_accuracy: 0.8650
Epoch 7/10
30/30 [==============================] - 443s 15s/step - loss: 0.3392 - accuracy: 0.8578 - val_loss: 0.3087 - val_accuracy: 0.8662
Epoch 